In [1]:
import os
import re
import nltk
from nltk.stem import PorterStemmer

In [2]:
# function to return a list of all files present in a directory (using os library)
def GenerateFilesList(Path):
    return os.listdir(Path + '/')

# function to extract all text in documents from a directory
def ExtractText(Path, FilesList):
    text=""
    for x in FilesList:
        with open(f"{Path}/{x}") as file:
            #opening every document and capturing its content
            file_text = file.read()
            text = text + file_text
    return text

In [3]:
# function to split the text into words and convert them all to lowercase (using re library)
# text --> lower case words
def SplitAndLowerText(text):
    #[^A-Za-z\'']+   --> regex for non-aplhabetic chars (and apostrophe) occuring 1 or more times [used as seperator for split]
    words = re.split('[^A-Za-z\']+',text)
    #converting all words to lower case to standardize
    lower_case_words=[word.lower() for word in words]
    return lower_case_words


# function to remove stopwords from list of words and return a list of cleaned words
# lower case words --> cleaned words
def StopwordsRemoval(words):
    import nltk
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    stopwords_nltk=set(stopwords.words('english'))
    cleaned_words=[word for word in words if word not in stopwords_nltk]
    return cleaned_words


# function to remove duplicates and return list of unique words
# cleaned words --> unique words
def RemoveDuplicates(words):
    unique_words=[]
    for word in words:
        if word not in unique_words:
            unique_words.append(word)
    return unique_words


# function to perform stemming on words list
# unique words --> stemmed words
def Stemmer(words):
    from nltk.stem import PorterStemmer
    stemmed_words=[]
    for word in words:
        stemmed_words.append(PorterStemmer().stem(word))
    return stemmed_words


In [4]:
# function to create dictionary of inverse index from words list
def GenerateDictionary(words, Path, FilesList):
    dict={}
    for word in words:
        dict[word]=[]
        
    for word in words:
        for x in range(len(FilesList)):
            with open(f"{Path}/{FilesList[x]}") as file:
                file_text = file.read()
                if word in file_text.lower():
                    dict[word].append(x)
    return dict

def GenerateDictionaryV2(UniqueWords, StemmedWords, Path, FilesList):
    dict={}
    for word in StemmedWords:
        dict[word]=[]
        
    for word in UniqueWords:
        for x in range(len(FilesList)):
            with open(f"{Path}/{FilesList[x]}") as file:
                file_text = file.read()
                if word in file_text.lower():
                    if x not in dict[PorterStemmer().stem(word)]:
                        dict[PorterStemmer().stem(word)].append(x)
    return dict

In [5]:
# function to return TRUE/FALSE for whether string matches with wildcard pattern or not
 
def StringMatch(strr, pattern, slen, plen):
 
    # empty pattern (plen=0) can only match with empty string
    if (plen == 0):
        return (slen == 0)
 
    # creating 2D array of 0-M X 0-N to store subsolutions 
    lookup = [[False for i in range(plen + 1)] for j in range(slen + 1)]
 
    # empty pattern matches with empty string
    lookup[0][0] = True
 
    # only '*' can match with empty string
    for j in range(1, plen + 1):
        if (pattern[j - 1] == '*'):
            lookup[0][j] = lookup[0][j - 1]
 
    # filling the table by following the rules below
    for i in range(1, slen + 1):
        for j in range(1, plen + 1):
 
            # if last char of pattern is '*': 
            # (a) either * is zero chars  
            #       --> solution for [i][j-1] is applicable
            # (b) * is a char \
            #       --> solution for [i-1][j] is applicable (since extra char from string is encorporated in *)
            if (pattern[j - 1] == '*'):
                lookup[i][j] = lookup[i][j - 1] or lookup[i - 1][j]
 
            # if last two chars of both are same or last char of pattern is '?', solution for [i-1][j-1] is applicable
            elif ( (pattern[j - 1] == '?') or (strr[i - 1] == pattern[j - 1]) ):
                lookup[i][j] = lookup[i - 1][j - 1]
                
            else:
                lookup[i][j] = False
 
    return lookup[slen][plen]

In [6]:
# function to return Edit Distance between two strings 
# [taking first string to be mutable, second string to be immutable]

def EditDistance(str1, str2, len1, len2):
    # creating 2D array of 0-len1 X 0-len2 to store subsolutions    
    dp = [[0 for x in range(len2 + 1)] for x in range(len1 + 1)]
 
    # filling 2D array 
    for i in range(len1 + 1):
        for j in range(len2 + 1):
 
            # if first string is empty, only way is to insert all characters of second string
            if i == 0:
                dp[i][j] = j    # Min. operations = j (length of second string)
 
            # if second string is empty, only way is to remove all characters of first string
            elif j == 0:
                dp[i][j] = i    # Min. operations = i (length of first string)
 
            # if last characters are same, then distance = d[i-1][j-1]
            elif str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
 
            # if last characters are different, consider insert(one char short)/remove(one extra char)/replace(diff char)
            else:
                dp[i][j] = 1 + min(dp[i][j-1],        # Insert
                                   dp[i-1][j],        # Remove
                                   dp[i-1][j-1])    # Replace
 
    return dp[len1][len2]

In [7]:
# function to spellcheck a word, returns word/closest word

def SpellCheck(word, VocabList):
    min = 10000
    closest = ""
    for x in VocabList:
        if (StringMatch(x, word, len(x), len(word))):
            return word
        else:
            distance = EditDistance(word, x, len(word), len(x))
            if (distance < min):
                min = distance
                closest = x
    return closest

In [8]:
# defining bitwise functions (using lists)

# bitwise AND
def BitwiseAND(list1, list2):
    bitwise_op = [w1 & w2 for (w1, w2) in zip(list1, list2)]
    return bitwise_op

# bitwise OR
def BitwiseOR(list1, list2):
    bitwise_op = [w1 | w2 for (w1, w2) in zip(list1, list2)]
    return bitwise_op

# bitwise NOT
def BitwiseNOT(list1, list2):
    bitwise_op = [not w1 for w1 in list2]
    bitwise_op = [int(b == True) for b in bitwise_op]
    bitwise_op = [w1 & w2 for (w1,w2) in zip(list1,bitwise_op)]
    return bitwise_op

In [9]:
# EXECUTION
def Execute(query_input):
    query = query_input.split()

    #boolean connectors
    connecting_words = []
    #content words
    different_words = []

    for word in query:
        if word.lower() != "and" and word.lower() != "or" and word.lower() != "not":
            different_words.append(word.lower())
        else:
            connecting_words.append(word.lower())

    zeroes_and_ones = []
    zeroes_and_ones_of_all_words = []

    for word in (different_words):
        #if word is normal query
        if word.isalpha():
            closest = SpellCheck(word, unique_words)
            stemmed = PorterStemmer().stem(closest)
            zeroes_and_ones = [0] * len(Files_List)
            for x in dictV2[stemmed]:
              zeroes_and_ones[x] = 1
            zeroes_and_ones_of_all_words.append(zeroes_and_ones)
        
        #if word in wildcary query
        else:
            matches = []
            for x in unique_words:
                if StringMatch(x, word, len(x), len(word)):
                    matches.append(x)

            stemmed_matches=[]
            for x in matches:
                stemmed = PorterStemmer().stem(x)
                if stemmed not in stemmed_matches:
                    stemmed_matches.append(stemmed)

            zeroes_and_ones = [0] * len(Files_List)
            temp_zeroes_and_ones = [0] * len(Files_List)
            for match in stemmed_matches:
                for x in dictV2[match]:
                    temp_zeroes_and_ones[x] = 1
                zeroes_and_ones = BitwiseOR(zeroes_and_ones, temp_zeroes_and_ones)
            zeroes_and_ones_of_all_words.append(zeroes_and_ones)


    for word in connecting_words:
        # we take the first two words from query (for which the corresponding connecting_word is taken)
        word_list1 = zeroes_and_ones_of_all_words[0]
        word_list2 = zeroes_and_ones_of_all_words[1]

        if word == "and":
            # performs bitwise operation AND on the two lists
            bitwise_op = BitwiseAND(word_list1,word_list2)
            zeroes_and_ones_of_all_words.remove(word_list1)
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op)
        elif word == "or":
            # performs bitwise operation OR on the two lists
            bitwise_op = BitwiseOR(word_list1,word_list2)
            zeroes_and_ones_of_all_words.remove(word_list1)
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op)
        elif word == "not":
            # performs bitwise operation NOT on the two lists
            bitwise_op = BitwiseNOT(word_list1,word_list2)
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.remove(word_list1)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op)
    # this results in a final list with bitwise operations done over the entire query

    files_found = []    
    if zeroes_and_ones_of_all_words:
        index_list = zeroes_and_ones_of_all_words[0]
    else:
        index_list = []

    for index in range(len(index_list)):
        if index_list[index]==1:
            files_found.append(Files_List[index])

    print("\n Files Found:",len(files_found))
    print("List of files found:\n", files_found)

In [10]:
# opening files and reading text
path = "dataset"
Files_List = GenerateFilesList(path)
text = ExtractText(path, Files_List)

# preprocessing
lower_case_words = SplitAndLowerText(text)
cleaned_words = StopwordsRemoval(lower_case_words)
unique_words = RemoveDuplicates(cleaned_words)
stemmed_words = Stemmer(unique_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# formulating inverse index
dict = GenerateDictionary(unique_words, path, Files_List)
dictV2 = GenerateDictionaryV2(unique_words, stemmed_words, path, Files_List)

In [12]:
Execute("androni?us")


 Files Found: 1
List of files found:
 ['titus-andronicus_TXT_FolgerShakespeare.txt']


In [15]:
Execute("andronics or lucetta")


 Files Found: 2
List of files found:
 ['the-two-gentlemen-of-verona_TXT_FolgerShakespeare.txt', 'titus-andronicus_TXT_FolgerShakespeare.txt']


In [14]:
Execute("andronics not titus")


 Files Found: 0
List of files found:
 []
